In [14]:
import pandas as pd
import numpy as np
import json
import csv

In [2]:
#1.确定这几个数据都是干什么的
swiping_card_dataframe = pd.read_csv('./bus_swiping_card_info1.csv')
card_info_dataframe = pd.read_csv('./card_info.csv')
bus_swip_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
bus_station_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')

In [3]:
#swiping_card_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间
#card_info_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间也是上面的信息，但是时间是去年的。
#用去年的时间的同比数据，去预测今年的同比数据，看是否一致，如果一致，说明经济发展没有变化，如果变大，说明经济好转
#bus_swip_dataframe ：一天的统计信息：各条线路、各个站点的刷卡数量信息
#bus_station_dataframe ： 站点的统计信息：各个站点的经纬度

In [12]:
station_distance_rank_index_list = np.loadtxt('./station_pair_distance_rank_index.csv',delimiter=',',dtype=int)

In [17]:
# 从文件中读取字典
with open('./id_2_station.json', 'r') as file:
    id_2_station = json.load(file)
# 将字典保存到文件中
with open('./station_2_id.json', 'r') as file:
    station_2_id = json.load(file)

In [20]:
#从card_info_dataframe中找出所有用户一天内的od对信息
card_info_dataframe['custom_day'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[0])
card_info_dataframe['custom_precise_time'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[1])
sum(card_info_dataframe['partitionday']==card_info_dataframe['custom_day'])

In [27]:
card_info_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,28,7520,54,2023-09-29,08:59:40
1,0x1FFC5D30D96AE2C4524842A09AA67C44,84,5085,24,2023-09-29,11:08:22
2,0x65F9DAD54040FBE48972B10F3B6337DC,25,6718,4,2023-09-29,08:51:28
3,0x747A26B4727A9A3094A1881F29D01510,208,6655,1,2023-09-29,22:32:48
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,204,4472,46,2023-09-29,19:50:37
...,...,...,...,...,...,...
724622,0x6B5CCBC949D06967421DDEFFA8682697,13,7618,22,2023-10-01,11:09:08
724623,0x7FF2BF197AE5BB5ECCBC9BC64FFEE659,7,5020,10,2023-10-01,15:34:58
724624,0x89C88A5125FCAC7607C396EDABD8FA50,320,4451,9,2023-10-01,10:13:20
724625,0xA159FB0957C071A17B4FC41C6F622556,61,3826,10,2023-10-01,17:22:23


In [24]:
card_info_dataframe.drop(labels=['custom_time','card_type','consume','data_src','data_load_time','partitionday'],axis=1,inplace=True)

In [29]:
day_0929_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-09-29']
day_0929_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,28,7520,54,2023-09-29,08:59:40
1,0x1FFC5D30D96AE2C4524842A09AA67C44,84,5085,24,2023-09-29,11:08:22
2,0x65F9DAD54040FBE48972B10F3B6337DC,25,6718,4,2023-09-29,08:51:28
3,0x747A26B4727A9A3094A1881F29D01510,208,6655,1,2023-09-29,22:32:48
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,204,4472,46,2023-09-29,19:50:37
...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,52,5054,22,2023-09-29,08:48:38
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,1,4662,6,2023-09-29,10:33:45
377161,0x6D191CC173F9415C1148B434D2F9012E,203,4810,5,2023-09-29,14:57:10
377162,0x7896A07D162530E734F876D16301EFEB,73,5176,23,2023-09-29,15:08:49


In [31]:
day_1001_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-10-01']
day_1001_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
377164,0x0C6B84DFFBEB7E3112481AE23499566B,222,7703,1,2023-10-01,08:33:23
377165,0x13A08316FB35BC0DFE0A98D00604E7E7,6,7744,4,2023-10-01,17:51:06
377166,0x1A146FC5EC5C78862B9A10C3004FCC2C,3,7477,15,2023-10-01,15:10:36
377167,0x3DC18BE83960BE730FC7CDCB33E1F2F5,226,7053,1,2023-10-01,15:25:31
377168,0x47602A7B7A8F237606D283798F822EEC,219,7158,21,2023-10-01,14:51:54
...,...,...,...,...,...,...
724622,0x6B5CCBC949D06967421DDEFFA8682697,13,7618,22,2023-10-01,11:09:08
724623,0x7FF2BF197AE5BB5ECCBC9BC64FFEE659,7,5020,10,2023-10-01,15:34:58
724624,0x89C88A5125FCAC7607C396EDABD8FA50,320,4451,9,2023-10-01,10:13:20
724625,0xA159FB0957C071A17B4FC41C6F622556,61,3826,10,2023-10-01,17:22:23


In [90]:
import heapq
# 要找出的最小的k个值
k = 3
# 使用列表推导式找出每行的最小的k个值
station_distance_k_values = [heapq.nsmallest(k, row) for row in station_pair_distance]

print(station_distance_k_values)

[[0.0, 321.81251849286264, 724.5687536525421], [0.0, 580.5780322709488, 682.3178681080469], [0.0, 176.91933282856345, 213.2404862913487], [0.0, 503.6789505716781, 534.4169974214412], [0.0, 209.05568164792004, 442.1250172769303], [0.0, 296.21522036957384, 383.51138986302936], [0.0, 4435088.475074348, 4435250.452118327], [0.0, 354.6858236746748, 555.1230557902543], [0.0, 14.886386699090393, 207.08484869433718], [0.0, 297.63156824634524, 636.3452050425832], [0.0, 398.0248514969028, 632.0148206737043], [0.0, 104.61891884172098, 133.632105665866], [0.0, 43.630723995853465, 104.61891884172098], [0.0, 43.630723995853465, 133.632105665866], [0.0, 286.60771430315674, 315.5605814385828], [0.0, 340.09463281064643, 355.93598799583236], [0.0, 281.3111787396845, 467.81530576598226], [0.0, 340.09463281064643, 432.8495548486716], [0.0, 232.25487924791304, 356.84214725062606], [0.0, 299.2912072045584, 317.6182000238952], [0.0, 1330.955616206553, 1380.397167007911], [0.0, 500.18443712070837, 608.2958651

In [102]:
# 将列表转换为NumPy数组
station_pair_distance_ndarray = np.array(station_pair_distance)
# 每行的最小的k个值所对应的index
k = 1  # 可以自定义k的值
station_pair_distance_min_k_indices = [np.argpartition(row, k)[:k] for row in station_pair_distance_ndarray]
print(station_pair_distance_min_k_indices)

[array([0]), array([1]), array([2]), array([3]), array([4]), array([5]), array([6]), array([7]), array([8]), array([9]), array([10]), array([11]), array([12]), array([13]), array([14]), array([15]), array([16]), array([17]), array([18]), array([19]), array([20]), array([21]), array([22]), array([23]), array([24]), array([25]), array([26]), array([27]), array([28]), array([29]), array([30]), array([31]), array([32]), array([33]), array([34]), array([35]), array([36]), array([37]), array([38]), array([39]), array([40]), array([41]), array([42]), array([43]), array([44]), array([45]), array([46]), array([47]), array([48]), array([49]), array([50]), array([51]), array([52]), array([53]), array([54]), array([55]), array([56]), array([57]), array([58]), array([59]), array([60]), array([61]), array([62]), array([63]), array([64]), array([65]), array([66]), array([67]), array([68]), array([69]), array([70]), array([71]), array([72]), array([73]), array([74]), array([75]), array([76]), array([7

In [105]:
# 找出每行最小的k个值所对应的索引
k=6
station_distance_min_indices = [sorted(range(len(row)), key=lambda i: row[i])[:k] for row in station_pair_distance]
station_distance_min_indices
# 按照值的大小对索引进行排序
#sorted_indices = sorted(range(len(data)), key=lambda i: data[i][min_indices[i][0]])

#print("按照最小值所对应的索引进行排序后的结果：", sorted_indices)

[[0, 536, 1259, 1226, 454, 977],
 [1, 361, 1225, 360, 362, 780],
 [2, 252, 1030, 704, 790, 15],
 [3, 381, 849, 925, 379, 380],
 [4, 1008, 265, 840, 841, 790],
 [5, 797, 222, 856, 855, 686],
 [6, 1384, 1183, 1383, 540, 906],
 [7, 637, 158, 1424, 17, 1425],
 [8, 206, 716, 164, 777, 468],
 [9, 1024, 259, 238, 304, 586],
 [10, 657, 1244, 1245, 1267, 324],
 [11, 12, 13, 462, 456, 870],
 [12, 13, 11, 462, 456, 870],
 [13, 12, 11, 462, 456, 870],
 [14, 915, 449, 69, 584, 447],
 [15, 17, 1030, 126, 1034, 252],
 [16, 1367, 1366, 755, 1358, 404],
 [17, 15, 637, 1030, 126, 252],
 [18, 578, 672, 674, 673, 19],
 [19, 673, 672, 674, 18, 578],
 [20, 103, 1202, 877, 593, 876],
 [21, 676, 853, 949, 855, 856],
 [22, 949, 1124, 1010, 1123, 853],
 [23, 1017, 745, 1018, 1073, 1074],
 [24, 625, 1339, 623, 1413, 353],
 [25, 710, 738, 658, 1139, 363],
 [26, 632, 916, 1188, 33, 1346],
 [27, 1278, 823, 824, 390, 1343],
 [28, 31, 594, 30, 29, 32],
 [29, 30, 375, 28, 31, 79],
 [30, 29, 375, 28, 31, 79],
 [31, 28,

In [108]:
station_pair_distance[0][536]

321.81251849286264

In [109]:
station_pair_distance[0][1259]

724.5687536525421

In [110]:
station_pair_distance[0][1226]

746.2495938000633

# 任务
1.从各个站点的经纬度出发，计算出每个站点距离其他站点的距离，结果大概为1000*1000的方阵，然后我们一般会选择前10个，结果为1000*10


In [84]:
from geopy import distance
#(纬度，经度)
point1 = (26.633052,106.628469)
point2 = (26.635156,106.617741)
dist = distance.distance(point1,point2).m
print('距离',dist)

距离 1093.3717847368966


In [3]:
sheet1 = pd.read_csv('./card_info.csv')

In [4]:
sheet1

,card_identity,card_type,custom_time,line_identity,car_identity,consume,station_identity,data_src,data_load_time,partitionday
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,公交云卡,2023-09-29 08:59:40,28,7520,1.8,54,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
1,0x1FFC5D30D96AE2C4524842A09AA67C44,成人卡,2023-09-29 11:08:22,84,5085,1.8,24,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
2,0x65F9DAD54040FBE48972B10F3B6337DC,通卡学生卡,2023-09-29 08:51:28,25,6718,0.0,4,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
3,0x747A26B4727A9A3094A1881F29D01510,学生卡,2023-09-29 22:32:48,208,6655,1.0,1,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,公交云卡,2023-09-29 19:50:37,204,4472,1.8,46,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
...,...,...,...,...,...,...,...,...,...,...
724622,0x6B5CCBC949D06967421DDEFFA8682697,通卡学生卡,2023-10-01 11:09:08,13,7618,0.0,22,黔中市公共交通(集团)有限公司,2024-01-18 15:16:23,2023-10-01
724623,0x7FF2BF197AE5BB5ECCBC9BC64FFEE659,公交云卡,2023-10-01 15:34:58,7,5020,1.8,10,黔中市公共交通(集团)有限公司,2024-01-18 15:16:23,2023-10-01
724624,0x89C88A5125FCAC7607C396EDABD8FA50,成人卡,2023-10-01 10:13:20,320,4451,0.0,9,黔中市公共交通(集团)有限公司,2024-01-18 15:16:23,2023-10-01
724625,0xA159FB0957C071A17B4FC41C6F622556,通卡学生卡,2023-10-01 17:22:23,61,3826,0.0,10,黔中市公共交通(集团)有限公司,2024-01-18 15:16:23,2023-10-01


In [47]:
day_0929 = sheet1[sheet1['partitionday']=='2023-09-29']
day_0929

,card_identity,card_type,custom_time,line_identity,car_identity,consume,station_identity,data_src,data_load_time,partitionday
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,公交云卡,2023-09-29 08:59:40,28,7520,1.8,54,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
1,0x1FFC5D30D96AE2C4524842A09AA67C44,成人卡,2023-09-29 11:08:22,84,5085,1.8,24,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
2,0x65F9DAD54040FBE48972B10F3B6337DC,通卡学生卡,2023-09-29 08:51:28,25,6718,0.0,4,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
3,0x747A26B4727A9A3094A1881F29D01510,学生卡,2023-09-29 22:32:48,208,6655,1.0,1,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,公交云卡,2023-09-29 19:50:37,204,4472,1.8,46,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
...,...,...,...,...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,通卡学生卡,2023-09-29 08:48:38,52,5054,0.0,22,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,通卡学生卡,2023-09-29 10:33:45,1,4662,0.0,6,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377161,0x6D191CC173F9415C1148B434D2F9012E,通卡学生卡,2023-09-29 14:57:10,203,4810,0.0,5,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377162,0x7896A07D162530E734F876D16301EFEB,公交云卡,2023-09-29 15:08:49,73,5176,1.8,23,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29


In [28]:
#先找出每个人的od对需要做两件事：
#1.确定每个公交线路上每个上车站沿着这个线路可达的站
#2.确定每个站点与周围站点的距离。
#起点站好确定，但是终点站不好确定，看这条线路跟他下一趟车上车位置距离最近的站是哪个，这个站就是他这趟路程的终点站。
#

,card_identity,card_type,custom_time,line_identity,car_identity,consume,station_identity,data_src,data_load_time,partitionday
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,公交云卡,2023-09-29 08:59:40,28,7520,1.8,54,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
1,0x1FFC5D30D96AE2C4524842A09AA67C44,成人卡,2023-09-29 11:08:22,84,5085,1.8,24,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
2,0x65F9DAD54040FBE48972B10F3B6337DC,通卡学生卡,2023-09-29 08:51:28,25,6718,0.0,4,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
3,0x747A26B4727A9A3094A1881F29D01510,学生卡,2023-09-29 22:32:48,208,6655,1.0,1,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,公交云卡,2023-09-29 19:50:37,204,4472,1.8,46,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
...,...,...,...,...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,通卡学生卡,2023-09-29 08:48:38,52,5054,0.0,22,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,通卡学生卡,2023-09-29 10:33:45,1,4662,0.0,6,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377161,0x6D191CC173F9415C1148B434D2F9012E,通卡学生卡,2023-09-29 14:57:10,203,4810,0.0,5,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29
377162,0x7896A07D162530E734F876D16301EFEB,公交云卡,2023-09-29 15:08:49,73,5176,1.8,23,黔中市公共交通(集团)有限公司,2023-11-15 10:50:11,2023-09-29


In [48]:
#公交站点数据：
sheet_bus

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
0,1,1,1,火车站,2023-10-01,17
1,1,1,2,展览馆,2023-10-01,318
2,1,1,3,新路口,2023-10-01,510
3,1,1,4,邮电大楼,2023-10-01,678
4,1,1,5,大十字,2023-10-01,598
...,...,...,...,...,...,...
6475,4005,0,10,南垭路口,2023-10-01,2
6476,4600,1,7,粑粑坳,2023-10-01,2
6477,4601,1,14,白金大道中,2023-10-01,1
6478,4601,1,16,刚玉街,2023-10-01,1


In [29]:
day_0929_card_identity=day_0929.groupby('card_identity').agg({'custom_time':list,'line_identity':list,'station_identity':list}).reset_index()

In [38]:
for time in list(zip(day_0929_card_identity[3:4]['custom_time'].values[0],day_0929_card_identity[3:4]['line_identity'].values[0],day_0929_card_identity[3:4]['station_identity'].values[0])):
    print(time)

('2023-09-29 09:26:05', 207, 14)
('2023-09-29 19:24:18', 202, 39)
('2023-09-29 16:29:50', 5, 5)
('2023-09-29 09:20:51', 204, 8)


In [39]:
sheet_bus[sheet_bus['线路编号']==204]['站点名称'].values[0]
#上车明秀宾馆，中途学士路口，回来皂角井站，大西门站

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
3093,204,1,1,洛平公交枢纽,2023-10-01,693
3094,204,1,2,明珠大道,2023-10-01,127
3095,204,1,3,西坝行政中心,2023-10-01,60
3096,204,1,8,明秀宾馆,2023-10-01,326
3097,204,1,9,西坝公园站,2023-10-01,128
...,...,...,...,...,...,...
3149,204,0,62,民族大学,2023-10-01,14
3150,204,0,63,贵州大学轨道站,2023-10-01,53
3151,204,0,64,贵州大学,2023-10-01,5
3152,204,0,68,红阳厂,2023-10-01,16


In [41]:
sheet_bus[sheet_bus['站点名称']=='学士路口']

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
3040,203,1,25,学士路口,2023-10-01,2
3197,207,1,14,学士路口,2023-10-01,135
3219,207,0,38,学士路口,2023-10-01,49
6204,809,1,12,学士路口,2023-10-01,2


In [45]:
sheet_bus[sheet_bus['线路编号']==202][sheet_bus[sheet_bus['线路编号']==202]['站点名称']=='明秀宾馆']

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
2957,202,1,5,明秀宾馆,2023-10-01,44


In [46]:
sheet_bus[sheet_bus['线路编号']==202]

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
2953,202,1,1,洛平公交枢纽,2023-10-01,5
2954,202,1,2,迎宾路口,2023-10-01,172
2955,202,1,3,阳光水乡,2023-10-01,10
2956,202,1,4,贵筑路口,2023-10-01,84
2957,202,1,5,明秀宾馆,2023-10-01,44
...,...,...,...,...,...,...
3011,202,0,60,西坝公园站,2023-10-01,17
3012,202,0,61,贵筑路口,2023-10-01,11
3013,202,0,62,阳光水乡,2023-10-01,47
3014,202,0,63,迎宾路口,2023-10-01,124


In [5]:
sheet2 = pd.read_csv('./bus_swiping_card_info1.csv')

In [10]:
sheet2[sheet2['car_identity']==5011]

,car_identity,card_type,custom_time,line_identity,card_identity,station_identity,partitionday
0,5011,公交云卡,2024-03-16 08:09:27,7,0x000229AC214F69A81F266A521394C628,33,2024-03-18
451,5011,公交云卡,2024-03-20 14:07:43,7,0x0065D26071147DE0BC86C8E64983FC74,36,2024-03-22
1459,5011,成人卡,2024-03-18 07:04:23,7,0x0126BD3073CD16D2F2E70A8351EBF7C6,1,2024-03-20
2181,5011,员工卡,2024-03-17 22:11:06,7,0x01C1A5494079A4081C02B4A32B3DB12E,19,2024-03-19
2568,5011,公交云卡,2024-03-19 06:49:30,7,0x01FF9B8D32C9257DDF14D5E4C8793AB5,6,2024-03-21
...,...,...,...,...,...,...,...
678528,5011,公交云卡,2024-03-17 07:13:01,7,0xFCC1209616E73D34895E155D15DFB9BB,8,2024-03-19
680707,5011,公交云卡,2024-03-19 07:40:42,7,0xFE4E222E0A38B9DBF9C617CAA5328328,30,2024-03-21
681425,5011,公交云卡,2024-03-18 22:21:18,7,0xFF0C6D1D29C9B45BA194E3D0AF2A68B3,11,2024-03-20
682344,5011,通卡学生卡,2024-03-19 17:56:35,7,0xFFC6895409A569630FBD2CAEC8D8A9A4,5,2024-03-21


In [14]:
sheet2['day'] = sheet2['custom_time'].apply(lambda x:x.split(' ')[0])

In [17]:
car_5011 = sheet2[sheet2['car_identity']==5011]

In [18]:
car_5011[car_5011['day']=='2024-03-17']

,car_identity,card_type,custom_time,line_identity,card_identity,station_identity,partitionday,day
2181,5011,员工卡,2024-03-17 22:11:06,7,0x01C1A5494079A4081C02B4A32B3DB12E,19,2024-03-19,2024-03-17
3697,5011,公交云卡,2024-03-17 20:48:46,7,0x02C3A8E96F1E3A8FFE0E7C3899028979,40,2024-03-19,2024-03-17
6999,5011,通卡学生卡,2024-03-17 17:51:47,7,0x054A41D277056350639E729668A96BBC,32,2024-03-19,2024-03-17
7002,5011,通卡学生卡,2024-03-17 17:51:46,7,0x054A41D277056350639E729668A96BBC,32,2024-03-19,2024-03-17
7017,5011,通卡学生卡,2024-03-17 13:25:26,7,0x055129A027AB92B0C516C7424E1D5282,43,2024-03-19,2024-03-17
...,...,...,...,...,...,...,...,...
656380,5011,通卡学生卡,2024-03-17 08:57:41,7,0xEC0A35EAC83E1DF003377C4679A00115,1,2024-03-19,2024-03-17
663993,5011,通卡学生卡,2024-03-17 10:47:35,7,0xF19185DBFE7086FB8FD178FE58442BE9,43,2024-03-19,2024-03-17
667366,5011,公交云卡,2024-03-17 14:33:15,7,0xF401F351EF1A83192A0F93A1E7B22166,7,2024-03-19,2024-03-17
670054,5011,通卡学生卡,2024-03-17 12:59:57,7,0xF60E0B2D83925AE551D81D4CC4EFC1F2,37,2024-03-19,2024-03-17


In [23]:
sheet2[sheet2['card_identity']=='0xFCC1209616E73D34895E155D15DFB9BB']

,car_identity,card_type,custom_time,line_identity,card_identity,station_identity,partitionday,day
678521,7639,公交云卡,2024-03-18 17:18:43,33,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-20,2024-03-18
678522,4422,公交云卡,2024-03-20 07:09:45,49,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-22,2024-03-20
678523,7531,公交云卡,2024-03-20 17:21:12,33,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-22,2024-03-20
678524,5013,公交云卡,2024-03-19 17:11:32,7,0xFCC1209616E73D34895E155D15DFB9BB,33,2024-03-21,2024-03-19
678525,4422,公交云卡,2024-03-19 07:09:18,49,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-21,2024-03-19
678526,5011,公交云卡,2024-03-16 07:23:20,7,0xFCC1209616E73D34895E155D15DFB9BB,8,2024-03-18,2024-03-16
678527,7649,公交云卡,2024-03-16 16:51:48,33,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-18,2024-03-16
678528,5011,公交云卡,2024-03-17 07:13:01,7,0xFCC1209616E73D34895E155D15DFB9BB,8,2024-03-19,2024-03-17
678529,5016,公交云卡,2024-03-17 16:21:17,7,0xFCC1209616E73D34895E155D15DFB9BB,33,2024-03-19,2024-03-17
678530,4422,公交云卡,2024-03-18 07:01:38,49,0xFCC1209616E73D34895E155D15DFB9BB,22,2024-03-20,2024-03-18


In [20]:
sheet_bus = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')

In [21]:
sheet_bus[sheet_bus['线路编号']==49]
#金顶路口	花果园金融街	花果园购物中心	花果园湿地公园	中山中路	遵义中路北
#龙洞堡客站

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
1643,49,1,1,阿哈湖湿地公园,2023-10-01,12
1644,49,1,2,延安南路口,2023-10-01,321
1645,49,1,3,公园中路南,2023-10-01,220
1646,49,1,4,公园中路,2023-10-01,203
1647,49,1,5,兰花广场,2023-10-01,74
1648,49,1,6,花果园大街,2023-10-01,116
1649,49,1,7,花果园大街中,2023-10-01,57
1650,49,1,8,花果园大街东,2023-10-01,131
1651,49,1,9,狮峰路口,2023-10-01,115
1652,49,1,10,浣沙桥,2023-10-01,97


In [22]:
sheet_bus[sheet_bus['线路编号']==33]
#金顶路口	花果园金融街	花果园购物中心	花果园湿地公园	中山中路	遵义中路北
#龙洞堡客站

,线路编号,方向,站点编号,站点名称,日期,刷卡数量
1034,33,1,1,贵大茶店校区,2023-10-01,1
1035,33,1,2,中天花园路口,2023-10-01,8
1036,33,1,3,茶店,2023-10-01,120
1037,33,1,4,计质检所,2023-10-01,271
1038,33,1,5,省二医,2023-10-01,5
1039,33,1,6,大营坡,2023-10-01,154
1040,33,1,7,贵开路,2023-10-01,143
1041,33,1,8,黔中医学院,2023-10-01,90
1042,33,1,9,北京路,2023-10-01,52
1043,33,1,10,北京西路口,2023-10-01,74


In [ ]:
#早上乘坐49号线路在22号站贵开路上车
#晚上乘坐33号线路在22号站湘雅村上车
#湘雅村后面的站有：【花果园，花香村，浣沙桥，黔灵山公园，北京西路口，北京路，黔中医学院，贵开路，大营坡，省二医，计质检所】
#贵开路后面的站有：【贵乌路，红边门，友谊路，黔灵西路，浣沙桥，花香村，狮峰路口，花果园大街东，花果园购物中心，】

In [ ]:
#湘雅村最近的站为狮峰路口，所以极有可能狮峰路口下车，湘雅村回去。

In [24]:
#周六周日
sheet_bus[sheet_bus['线路编号']==7]
#工作日


,线路编号,方向,站点编号,站点名称,日期,刷卡数量
166,7,1,1,世纪园小区,2023-10-01,18
167,7,1,2,丽景阳天,2023-10-01,175
168,7,1,3,小石城二期,2023-10-01,202
169,7,1,4,小石城,2023-10-01,141
170,7,1,5,春雷水库,2023-10-01,77
171,7,1,6,沙河花园,2023-10-01,163
172,7,1,7,鹿冲关路,2023-10-01,217
173,7,1,8,贵开路,2023-10-01,183
174,7,1,9,贵州医科大学,2023-10-01,13
175,7,1,10,六广门,2023-10-01,91
